In [3]:
import torch
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [2]:
dataset = Dataset.load_from_disk("../../02_实战演练篇/09_对话机器人/alpaca_data_zh") 
dataset
dataset[0:3]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("D:/Study_Date/Modelscope/cache/modelscope/Llama-2-7b-ms")
tokenizer

In [4]:
tokenizer.padding_side = "right"
tokenizer.pad_token_id = 2
tokenizer

In [5]:
def preprocess_function(examples):
    max_length = 1024
    inputs_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + examples["instruction"], examples["input"]]).strip() + "\n\nAssistant: ",add_special_tokens=False)
    response = tokenizer(examples["output"], add_special_tokens=False)
    inputs_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id] # 模型只需要学习response
    if len(inputs_ids) > max_length:
        inputs_ids = inputs_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    return {"input_ids": inputs_ids, "attention_mask": attention_mask, "labels": labels}

In [6]:
tokenized_datasets = dataset.map(preprocess_function,remove_columns=dataset.column_names)
tokenized_datasets

In [7]:
tokenizer.decode(tokenized_datasets[0]["input_ids"])

In [5]:
model = AutoModelForCausalLM.from_pretrained("D:/Study_Date/Modelscope/cache/modelscope/Llama-2-7b-ms",low_cpu_mem_usage=True,torch_dtype=torch.float16)

In [4]:
for name, param in model.named_parameters():
    print(name, param.dtype)

In [10]:
config = LoraConfig(task_type=TaskType.CAUSAL_LM)
config

In [11]:
model = get_peft_model(model, config)
config
for name, param in model.named_parameters():
    print(name, param.dtype)

In [12]:
model.enable_input_require_grads()# 开启梯度检查点时候必须要开启这个选项
model = model.half()
for name, param in model.named_parameters():
    print(name, param.dtype)

In [13]:
args = TrainingArguments(
    output_dir="./01_半精度训练",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    save_steps=20,
    gradient_checkpointing=True,
    adam_epsilon=1e-4,
)

trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets.select(range(30)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [14]:
trainer.train()